In [1]:
import logging
import boto3
import os
import json
import time
import zipfile
import subprocess
from textwrap import dedent

In [2]:
# -----------------------------------------------------------------------------
# Configure Logging
# -----------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [3]:
from src.utils.bedrock_agent import Agent, SupervisorAgent, agents_helper, region, account_id

2025-02-09 17:49:11,217 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


boto3 version: 1.36.9


In [4]:
sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region_name=region)
bedrock_client = boto3.client('bedrock-runtime', region_name=region)
iam_client = boto3.client('iam', region_name=region)
lambda_client = boto3.client('lambda', region_name=region)

logger.info(f"Region: {region}")
logger.info(f"Account ID: {account_id}")
logger.info(f"Agent Suffix: {agent_suffix}")


2025-02-09 17:49:12,694 - __main__ - INFO - Region: us-west-2
2025-02-09 17:49:12,694 - __main__ - INFO - Account ID: 533267284022
2025-02-09 17:49:12,695 - __main__ - INFO - Agent Suffix: us-west-2-533


In [16]:
# Agent foundation model 
agent_foundation_model = [
    # "anthropic.claude-3-5-sonnet-20241022-v2:0"
    "us.amazon.nova-lite-v1:0"
]

# Force re-create default setting for Agent objects, but for now set to False
Agent.set_force_recreate_default(True)

In [17]:
reasoning_query_lambda_arn = "arn:aws:lambda:us-west-2:533267284022:function:lambda-to-invoke-deepseek-r1-distilled-model"

In [18]:
reasoning_agent = Agent.direct_create(
    name=f"deepseek-agent-{agent_suffix}",
    role="Reasoning Specialist",
    goal="Analyze complex scenarios and provide well-reasoned answers",
    instructions="""
    You are a reasoning specialist tasked with analyzing complex scenarios and providing well-reasoned answers.

    ## Output Format:
    - Explain your reasoning and thought process in a <reasoning>...</reasoning> block
    - Include any assumptions made in your analysis in a <assumptions>...</assumptions> block
    """,
    tool_code=reasoning_query_lambda_arn,
    tool_defs=[
        {
            "name": "execute_reasoning",
            "description": "Executes a reasoning model to analyze complex scenarios",
            "parameters": {
                "scenario": {
                    "description": "The scenario to analyze",
                    "type": "string",
                    "required": True
                }
            }
        }
    ]

)


Deleting existing agent and corresponding lambda for: deepseek-agent-us-west-2-533...
Agent deepseek-agent-us-west-2-533 not found
Creating agent deepseek-agent-us-west-2-533...
Created agent, id: RMXZVL9BRC, alias id: TSTALIASID

Adding action group with Lambda: arn:aws:lambda:us-west-2:533267284022:function:lambda-to-invoke-deepseek-r1-distilled-model...
Waiting for agent status to change. Current status CREATING
Agent id RMXZVL9BRC current status: NOT_PREPARED
Waiting for agent status to change. Current status VERSIONING
Agent id RMXZVL9BRC current status: PREPARED
DONE: Agent: deepseek-agent-us-west-2-533, id: RMXZVL9BRC, alias id: JMSVNYPJ8M



In [29]:
REASONING_QUERY_LAMBDA_NAME = "lambda-to-invoke-deepseek-r1-distilled-model"
reasoning_agent_id = agents_helper.get_agent_id_by_name(reasoning_agent.name)

In [25]:

def add_resource_based_policy(function_name: str,
                              agent_ids: list,
                              region_name: str,
                              account_id: str):
    """
    Adds a resource-based policy to the specified Lambda function to allow invocation
    from one or more Bedrock agents.

    :param function_name: Name of the Lambda function.
    :param agent_ids: List of agent IDs permitted to invoke this Lambda.
    :param region_name: AWS region.
    :param account_id: AWS account ID.
    """
    logger.info(f"Adding resource-based policy to Lambda function {function_name} for agents: {agent_ids}")
    statement_id_prefix = "AllowExecutionFromBedrockAgent"
    policy_doc = {
        "Version": "2012-10-17",
        "Statement": []
    }

    for agent_id in agent_ids:
        sid = f"{statement_id_prefix}_{agent_id}"
        policy_doc['Statement'].append({
            "Sid": sid,
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "lambda:InvokeFunction",
            "Resource": f"arn:aws:lambda:{region_name}:{account_id}:function:{function_name}",
            "Condition": {
                "ArnLike": {
                    "AWS:SourceArn": f"arn:aws:bedrock:{region_name}:{account_id}:agent/{agent_id}"
                }
            }
        })

    # Retrieve existing policy and remove any existing statements with the same prefix
    try:
        existing_policy = lambda_client.get_policy(FunctionName=function_name)
        existing_policy_doc = json.loads(existing_policy['Policy'])
        for stmt in existing_policy_doc['Statement']:
            if stmt['Sid'].startswith(statement_id_prefix):
                sid_to_remove = stmt['Sid']
                logger.info(f"Removing existing statement: {sid_to_remove}")
                lambda_client.remove_permission(
                    FunctionName=function_name,
                    StatementId=sid_to_remove
                )
    except lambda_client.exceptions.ResourceNotFoundException:
        logger.info(f"No existing policy found for Lambda function {function_name}.")
    except Exception as e:
        logger.error(f"Error retrieving/removing existing policy for {function_name}: {str(e)}")

    # Add new permissions
    for stmt in policy_doc['Statement']:
        sid_val = stmt['Sid']
        try:
            lambda_client.add_permission(
                FunctionName=function_name,
                StatementId=sid_val,
                Action=stmt['Action'],
                Principal=stmt['Principal']['Service'],
                SourceArn=stmt['Condition']['ArnLike']['AWS:SourceArn']
            )
            logger.info(f"Added permission for statement: {sid_val}")
        except Exception as e:
            logger.error(f"Failed to add resource-based policy for {function_name}, statement {sid_val}: {str(e)}")

In [26]:
add_resource_based_policy(REASONING_QUERY_LAMBDA_NAME, [reasoning_agent_id], region, account_id)

2025-02-10 09:05:09,389 - __main__ - INFO - Adding resource-based policy to Lambda function lambda-to-invoke-deepseek-r1-distilled-model for agents: ['RMXZVL9BRC']
2025-02-10 09:05:09,439 - __main__ - INFO - Removing existing statement: AllowExecutionFromBedrockAgent_RMXZVL9BRC
2025-02-10 09:05:09,602 - __main__ - INFO - Added permission for statement: AllowExecutionFromBedrockAgent_RMXZVL9BRC


In [27]:
user_query = "What is the best way to reduce carbon emissions?"

In [30]:
import uuid

response = reasoning_agent.invoke(
    input_text=user_query,
    session_id=uuid.uuid4().hex,
    enable_trace=True,
    trace_level="core")

invokeAgent API request ID: 686577e4-7a2e-4c88-8c69-217f0067a28e
invokeAgent API session ID: 85282b44d3d647eb91a45d29b3d5f79f
  agent id: RMXZVL9BRC, agent alias id: TSTALIASID
---- Step 1 ----
Took 3.8s, using 835 tokens (in: 687, out: 148) to complete prior action, observe, orchestrate.
To answer this question about reducing carbon emissions, I'll need to analyze the complex scenario of global carbon emissions and potential reduction strategies. I'll use the reasoning model to help provide a well-reasoned answer.
Using tool: execute_reasoning with these inputs:
[{'name': 'scenario', 'type': 'string', 'value': 'What are the most effective strategies for reducing carbon emissions on a global scale, considering various sectors such as energy, transportation, industry, and agriculture?'}]

--tool outputs:
{'TEXT': {'body': {'generation': "What are the most effective strategies for reducing carbon emissions on a global scale, considering various sectors such as energy, transportation, ind